In [34]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import uuid
es = Elasticsearch(['http://localhost:9200/'], verify_certs=True)

if not es.ping():
    raise ValueError("Connection failed")

In [38]:
import jsonlines
from tqdm import tqdm 
import re 

list_a = []

root_path = "/data/szr207/dataset/ArqMath/jsons/answers/"

with jsonlines.open(os.path.join(root_path,'all.ans.jsonl')) as reader:
        for obj in tqdm(reader):
            if obj['body']:
                obj['body'] = re.sub('<[^<]+?>', '',  obj['body'])
            dict_item = {
                    "_index": "answer_bulk_index",
                    "_id": uuid.uuid4(),
                    "_source": obj }
            list_a.append(dict_item)
helpers.bulk(es, list_a)

1435643it [01:15, 18897.39it/s]


(1435643, [])

In [39]:
!jupyter nbconvert --to script create_pairwise.ipynb

[NbConvertApp] Converting notebook create_pairwise.ipynb to script
[NbConvertApp] Writing 8281 bytes to create_pairwise.py


In [1]:
import re
import jsonlines
import re 
import os 
from tqdm.notebook import tqdm

ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

list_raw = []

with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            list_raw.append(re.sub('<[^<]+?>', '',obj['body']))


with jsonlines.open(os.path.join(ques_path)) as reader:
        for obj in tqdm(reader):
            list_raw.append(re.sub('<[^<]+?>', '',obj['title']+ ' ' +obj['body']))

In [26]:
from nltk.tokenize import sent_tokenize, word_tokenize
import operator
from functools import reduce

MIN_TOKENS = 200 
MAX_TOKENS = 500


def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

def length_normalize_check(text):
#     all_words = word_tokenize(text)
    sent_tokens = sent_tokenize(text)
    list_sent = []
    
    list_sentences = list(divide_chunks(sent_tokens, 10))
    for sent in list_sentences:
        list_sent.append(' '.join(sent))
#     return reduce(operator.concat, list_sentences)
    return list_sent
    

In [6]:
with open('/data/szr207/dataset/ArqMath/jsons/arqmath.raw', 'w') as filehandle:
    for listitem in tqdm(list_raw):
        filehandle.write('%s\n' % listitem)

In [2]:
!pip install nltk

  Using cached nltk-3.5.zip (1.4 MB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached joblib-0.15.1-py3-none-any.whl (298 kB)
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=d516350a4f59baf05096e5bdf93abeced90512228ed7c841c5d7a06d6a164754
  Stored in directory: /home/szr207/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


In [30]:
final_raw = []

for raw in tqdm(list_raw):
    for sent in length_normalize_check(raw):
        final_raw.append(sent)
    

In [31]:
with open('/data/szr207/dataset/ArqMath/jsons/arqmath.raw', 'w') as filehandle:
    for listitem in tqdm(final_raw):
        filehandle.write('%s\n' % listitem)

In [10]:
len(all_words)

637

In [29]:
len(word_tokenize('Remember that whatever method you use for solving y^{\\prime}=f(t,y), be it Runge-Kutta, Bulirsch-Stoer (extrapolative), Gear/Adams multistep, or fancier methods, one always has a triple of values (t_i,y_i,y_i^{\\prime}) (y_i^{\\prime}=f(t_i,y_i)) at each evaluation point. Thus, one can always do cubic Hermite interpolation across the points (t_i,y_i) and (t_{i+1},y_{i+1}) (note that I am not assuming that the evaluation points are equispaced, as is often the case when doing adaptive solving). If the underlying method is at most third order accurate, cubic Hermite is a good choice. Now, the upshot is that modern implementations of DE solvers always support so-called "dense output"; briefly, associated with a method with $p$th order "accuracy" (in quotes since "high order doesn\'t always imply high accuracy" ;) ) has with it an associated $p$th order interpolating function. To use my favorite example, the (4,5) adaptive Runge-Kutta solver based on the Dormand-Prince coefficients has with it an associated fifth order interpolating function. The special properties inherent in the coefficients allow the existence of an associated interpolating function with the same order of "accuracy"; in general, not all Runge-Kutta coefficients will have an associated "nice" interpolating function (but again, one can always do cubic Hermite). I could say more, but the books by Hairer/Nørsett/Wanner have an extensive discussion on dense output (and they say it better than what I can hope to say), as well as usable routines (also available within the site). You would do well to study them.'))

338

In [34]:
!pip upgrade

^C


In [35]:
pip upgrade

^C
Note: you may need to restart the kernel to use updated packages.


In [38]:
from transformers import RobertaTokenizer, AutoModel
import torch
from torch import nn

tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained("shauryr/checkpoint-475000")

In [39]:
input_ids = torch.tensor([tokenizer.encode("\\Omega(k \\log k)")])
output_ids = torch.tensor([tokenizer.encode("\\Omega(m \\log p)")])

In [40]:
input_states, all_attentions = model(input_ids)[-2:]
output_states, all_attentions = model(output_ids)[-2:]

In [55]:
input_states[0].reshape(1,768)

RuntimeError: shape '[1, 768]' is invalid for input of size 8448

In [51]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos(input_states[0][5].reshape(1,768), output_states[0][5].reshape(1,768))

tensor([0.8117], grad_fn=<DivBackward0>)

In [57]:
model(input_ids)[1].shape

torch.Size([1, 768])

## TRY language masking

Comparing the default BERT model and RoBERTa-base fine-tuned on ARQMath dataset. These 2 are actual formula with a <mask> token in them. We can clearly see that the fine-tuned models are performing better than the general purpose BERT model

In [89]:
from transformers import pipeline
nlp_pipe_math = pipeline("fill-mask", model="shauryr/arqmath-roberta-base", tokenizer='roberta-base')
nlp_pipe_default = pipeline("fill-mask") #default BERT

In [90]:
nlp_pipe_math('\\Omega(k \\<mask> k)')

[{'sequence': '<s> \\Omega(k \\log k)</s>',
  'score': 0.7194331288337708,
  'token': 12376},
 {'sequence': '<s> \\Omega(k \\ln k)</s>',
  'score': 0.09566952288150787,
  'token': 8502},
 {'sequence': '<s> \\Omega(k \\times k)</s>',
  'score': 0.07879886031150818,
  'token': 14616},
 {'sequence': '<s> \\Omega(k \\over k)</s>',
  'score': 0.02194512076675892,
  'token': 2137},
 {'sequence': '<s> \\Omega(k \\mod k)</s>',
  'score': 0.011039352975785732,
  'token': 14377}]

In [91]:
nlp_pipe_default('\\Omega(k \\<mask> k)')

[{'sequence': '<s> \\Omega(k \\Delta k)</s>',
  'score': 0.4043600261211395,
  'token': 38816},
 {'sequence': '<s> \\Omega(k \\lambda k)</s>',
  'score': 0.059202104806900024,
  'token': 49744},
 {'sequence': '<s> \\Omega(k \\partial k)</s>',
  'score': 0.05178029462695122,
  'token': 45593},
 {'sequence': '<s> \\Omega(k \\quad k)</s>',
  'score': 0.02363007143139839,
  'token': 41666},
 {'sequence': '<s> \\Omega(k \\alpha k)</s>',
  'score': 0.01644393987953663,
  'token': 29135}]

In [92]:
nlp_pipe_default('0.999999999<mask>=1')

[{'sequence': '<s> 0.9999999990000000=1</s>',
  'score': 0.22802169620990753,
  'token': 45121},
 {'sequence': '<s> 0.999999999000000=1</s>',
  'score': 0.1923193782567978,
  'token': 33413},
 {'sequence': '<s> 0.999999999999=1</s>',
  'score': 0.08729714155197144,
  'token': 16692},
 {'sequence': '<s> 0.9999999990000=1</s>',
  'score': 0.03466024994850159,
  'token': 14200},
 {'sequence': '<s> 0.999999999 BTC=1</s>',
  'score': 0.01751110330224037,
  'token': 7432}]

In [93]:
nlp_pipe_math('0.999999999<mask>=1')

[{'sequence': '<s> 0.999999999...=1</s>',
  'score': 0.6985778212547302,
  'token': 734},
 {'sequence': '<s> 0.999999999....=1</s>',
  'score': 0.07275453954935074,
  'token': 17220},
 {'sequence': '<s> 0.999999999.....=1</s>',
  'score': 0.014931757934391499,
  'token': 38917},
 {'sequence': '<s> 0.99999999999=1</s>',
  'score': 0.014026593416929245,
  'token': 2831},
 {'sequence': '<s> 0.999999999999=1</s>',
  'score': 0.013251962140202522,
  'token': 16692}]

In [88]:
nlp_pipe('0.999999999<mask>=1')

[{'sequence': '<s> 0.9999999990000000=1</s>',
  'score': 0.22802169620990753,
  'token': 45121},
 {'sequence': '<s> 0.999999999000000=1</s>',
  'score': 0.1923193782567978,
  'token': 33413},
 {'sequence': '<s> 0.999999999999=1</s>',
  'score': 0.08729714155197144,
  'token': 16692},
 {'sequence': '<s> 0.9999999990000=1</s>',
  'score': 0.03466024994850159,
  'token': 14200},
 {'sequence': '<s> 0.999999999 BTC=1</s>',
  'score': 0.01751110330224037,
  'token': 7432}]

In [70]:
nlp_qa = pipeline('question-answering', model="shauryr/arqmath-roberta-base", tokenizer='roberta-base')

{'score': 1.3738734686305942e-05,
 'start': 32,
 'end': 50,
 'answer': '0.999999999\\dots=1'}

In [ ]:
"I'm told by smart people that 0.999999999\\dots=1 and I believe them, but is there a proof that explains why this is?"

In [71]:
nlp_qa(context='I was told by smart people that 0.999999999\\dots=1 and I believe them', question='proof that explains why this is?')

{'score': 1.3738734686305942e-05,
 'start': 32,
 'end': 50,
 'answer': '0.999999999\\dots=1'}

In [ ]:
"Remember that whatever method you use for solving (t_i,y_i,y_i^{\\prime}) (y_i^{\\prime}=f(t_i,y_i)) at each evaluation point."

In [74]:
nlp_qa(context="Remember that whatever method you use for solving <span class=\"math-container\" id=\"154115\">y^{\\prime}=f(t,y)</span>, be it Runge-Kutta, Bulirsch-Stoer (extrapolative), Gear/Adams multistep, or fancier methods, one always has a triple of values <span class=\"math-container\" id=\"154116\">(t_i,y_i,y_i^{\\prime})</span> (<span class=\"math-container\" id=\"154117\">y_i^{\\prime}=f(t_i,y_i)</span>) at each evaluation point. Thus, one can always do cubic Hermite interpolation across the points <span class=\"math-container\" id=\"154118\">(t_i,y_i)</span> and <span class=\"math-container\" id=\"154119\">(t_{i+1},y_{i+1})</span> (note that I am not assuming that the evaluation <span class=\"math-container\" id=\"154120\">p</span>oints are equis<span class=\"math-container\" id=\"154121\">p</span>aced, as is often the case when doing adaptive solving). If the underlying method is at most third order accurate, cubic Hermite is a good choice.</p>  <p>Now, the upshot is that modern implementations of DE solvers always support so-called \"dense output\"; briefly, associated with a method with $p$th order \"accuracy\" (in quotes since \"high order doesnt always imply high accuracy\" ;) ) has with it an associated $p$th order interpolating function. To use my favorite example, the <span class=\"math-container\" id=\"154122\">(4,5)</span> adaptive Runge-Kutta solver based on the Dormand-Prince coefficients has with it an associated fifth order interpolating function. The special properties inherent in the coefficients allow the existence of an associated interpolating function with the same order of \"accuracy\"; in general, not all Runge-Kutta coefficients will have an associated \"nice\" interpolating function (but again, one can always do cubic Hermite). I could say more, but the books by Hairer/Nørsett/Wanner</a> have an extensive discussion on dense output (and they say it better than what I can hope to say), as well as usable routines (also available within the site). You would do well to study them.", question="What is Hermite interpolation ?")

{'score': 3.063697650796922e-05,
 'start': 1432,
 'end': 1480,
 'answer': 'function. The special properties inherent in the'}

In [82]:
context = "The main idea of Hermite is to take x1→x0, but for this to succeed you cannot use the standard Lagrange basis L0(x):=(x−x1)/(x0−x1) and L1(x):=(x−x0)/(x1−x0) because the denominators will go to zero while the numerators stay finite.So the trick is to replace Lagrange basis with another one (there's nothing really special about Lagrange basis except that it makes collocation easy). For example take M0(x):≡1 and M1(x):=(x−x0) (which happens to be the naive Vandermonde basis when x0=0 but consider that an accident). Look now for two coefficients, say c0,c1 such that the linear function c0M0+c1M1 interpolates f at x0,x1, you solve a linear system and you'll getc0=f(x0) and c1=f(x1)−f(x0)x1−x0.(Can you smell Newton's divided differences? Never mind, focus on Hermite for now.) So in fact, posing h=x1−x0 we see that the only quantity that depends on h is c1=c^1(h), while c0,M0,M1 are h-independent, so we have an h-dependent Lagrange interpolant L(h)(x)=c0M0(x)+c^1(h)M1(x). This is useful. Drums rolling, we let x1→x0 (or h→0) and track the quantities that depend on h, i.e., c^1(h) which will simply converge to f′(x0), hence the linear interpolant becomes"
question = "what is the standard Lagrange basis?"

In [83]:
nlp_qa(context=context, question=question)

{'score': 2.9588249041183172e-05,
 'start': 279,
 'end': 337,
 'answer': "another one (there's nothing really special about Lagrange"}

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)

if not es.ping():
    raise ValueError("Connection failed")

In [8]:
query = "The purpose of the present invention is provide a slidable adhesive tape that allows the amount of outgassing, particularly of siloxane gas, to be greatly reduced. A slidable adhesive tape (6) comprises an adhesive layer (2) and a peelable liner (3) on one side of a base material (1), and is characterized in that the base material (1) comprises a fluororesin film or a polyetheretherketone film and the peelable liner (3) comprises a non-silicone-based peelable liner."

body = {
            "size": 5,
             "query": {
                   "more_like_this" : {
                "fields" : ["paperAbstract", "title"],
                "like" : query,
            }
            }
        }

res = es.search(index="opencorpus", body=body, request_timeout=1000)

In [9]:
res

{'took': 106293,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 111.033104,
  'hits': [{'_index': 'opencorpus',
    '_type': '_doc',
    '_id': '9nGdbHIBuT-Ur5XV2qv-',
    '_score': 111.033104,
    '_source': {'entities': [],
     'journalVolume': '',
     'journalPages': '',
     'pmid': '',
     'fieldsOfStudy': ['Materials Science'],
     'year': 2013,
     'outCitations': [],
     's2Url': 'https://semanticscholar.org/paper/58d447d60c4af70694d63531feef95ea55abdba2',
     's2PdfUrl': '',
     'id': '58d447d60c4af70694d63531feef95ea55abdba2',
     'authors': [{'name': '博之 飯田', 'ids': ['87123418']},
      {'name': '秋桐 村田', 'ids': ['116904390']}],
     'journalName': '',
     'paperAbstract': 'The purpose of the present invention is provide a slidable adhesive tape that allows the amount of outgassing, particularly of siloxane gas, to be greatly reduced. A slidable adhesiv

In [72]:
import xml.etree.ElementTree as ET
from elasticsearch import Elasticsearch
import re

def remove_stop(query):
    with open('../englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query


def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct


es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)

# query = "How to compute this combinatoric sum?  I have the sum \sum_{k=0}^{n} \binom{n}{k} k I know the result is n 2^{n-1} but I don\'t know how you get there. How does one even begin to simplify a sum like this that has binomial coefficients."
query = "How to compute this combinatoric sum?  "
# I have the sum \sum_{k=0}^{n} \binom{n}{k} k I know the result is n 2^{n-1} but I don\'t know how you get there. How does one even begin to simplify a sum like this that has binomial coefficients."
# query = re.sub('<[^<]+?>', '', query)
# query = query.lower()
# query = remove_stop(query)
# query = remove_punct(query)

# body = {
#             "size": 10,
#                      "query": {
#                     "multi_match": {
#                             "query": query,
#                             "fields":  ['title^3',"body"],
#                             "type": "cross_fields"
#                         }
#                      }}  
body = {
        "size": 10,
        "query": {
               "more_like_this" : {
            "fields" : ["title"],
            "like" : query,
            "min_term_freq" : 1,
            "max_query_terms" : 200
        }}}

res = es.search(index="question_bulk_index", body=body)
count = 1
for result in res['hits']['hits'][:10]:
    print(str(result['_source']['post_id'])+'\t'+str(count)+'\t'+ str(result['_score'])+ '\t'+ result['_source']['title'] +'\t'+ result['_source']['body']+'\n')

72618	1	20.81068	How to compute this sum	I'm trying to remove the summation sign from this formula, is this possible?  1+\sum_{k=1}^{500}(4(2k+1)^2-12k) 

923613	1	20.81068	How to compute this sum?	I want to sum the following:  f(n) = \sum_{i=1}^n (i^3 \cdot (n \mod i))  Since the sum can be huge I have to output the sum modulo some given number m. How can I approach this problem?  Also, n can be upto 10^{10}. 

2394328	1	19.80419	How to compute this infinite sum?	I'm trying to compute the infinite sum  \sum_{n=1}^{\infty}n(\frac{1}{2})^n  which I believe should represent the expected amount of coin flips needed to get a head. Can someone remind me how to do this? 

1386906	1	18.888252	Can this combinatoric sum be simplified?	Base cases:  F(n,k,d) = 0 if d=0 and n&gt;0  F(n,k,d) = 1 if n=0  Expression:  F(n,k,d) = \sum_{s=0}^{\min(k,n)}\binom{n}{s}F(n-s,k,d-1)  I am trying to compute the value of F(n,k,10) where n is a large positive integer and k is a positive integer (can be small or

In [73]:
# %%capture
# !pip install pyserini==0.8.1.0
# !pip install transformers

import json
import os
import torch
import numpy
from tqdm import tqdm
from transformers import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-11.0.7.10-4.el7_8.x86_64"

topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Sample Topics/Task1_Samples_V2.0.xml"
model_path = "/data/szr207/github/transformers/examples/language-modeling/output/checkpoint-3000000"
tokenizer = AutoTokenizer.from_pretrained('roberta-base', do_lower_case=False)
model = AutoModel.from_pretrained(model_path)query = ("How to compute this combinatoric sum?. I have the sum  $$\\sum_{k=0}^{n} \\binom{n}{k} k$$  I know the result is $n 2^{n-1}$ but I don't know how you get there. How does one even begin to simplify a sum like this that has binomial coefficients. ")

In [81]:
def extract_bert(text, tokenizer, model):
    text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    text_words = tokenizer.convert_ids_to_tokens(text_ids[0])[1:-1]

    n_chunks = int(numpy.ceil(float(text_ids.size(1))/510))
    states = []
    
    for ci in range(n_chunks):
        text_ids_ = text_ids[0, 1+ci*510:1+(ci+1)*510]            
        text_ids_ = torch.cat([text_ids[0, 0].unsqueeze(0), text_ids_])
        if text_ids[0, -1] != text_ids[0, -1]:
            text_ids_ = torch.cat([text_ids_, text_ids[0,-1].unsqueeze(0)])
        
        with torch.no_grad():
            state = model(text_ids_.unsqueeze(0))[0]
            state = state[:, 1:-1, :]
        states.append(state)

    state = torch.cat(states, axis=1)
    return text_ids, text_words, state[0]

In [82]:
text_ids, query_words, query_state = extract_bert(query, tokenizer, model)

In [91]:
query_state

torch.Size([78, 768])

In [99]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -q -m ndcg /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv /data/szr207/projects/ArqMath/runs/es.fintune.roberta-base.dat

ndcg                  	A.101	0.1568
ndcg                  	A.31	0.1602
ndcg                  	A.78	0.0443
ndcg                  	all	0.1204


### get vectors for all formulae and see what finetuned model returns 

trying see to see if BERT even learns something about formulae or not ?

In [1]:
import glob, os
import pandas as pd
from tqdm.notebook import tqdm 
import numpy as np
from sklearn.neighbors import NearestNeighbors
from transformers import pipeline
from transformers import *
import math
from IPython.core.display import display, HTML

latex_root = '/data/szr207/dataset/ArqMath/latex_representation/'


# os.chdir("latex_root")
list_tsv = []
for file in glob.glob(latex_root+"*.tsv"):
    list_tsv.append(file)

In [2]:
list_formula = []
for tsv in tqdm(list_tsv):
    df = pd.read_csv(tsv, sep="\t")
    df.dropna()
    list_formula.extend(df['formula'])

In [3]:
len(list_formula)

list_formula = list(set(list_formula))
list_formula = [x for x in list_formula if str(x) != 'nan']

In [4]:
feat_ext = pipeline("feature-extraction", model="/data/szr207/github/transformers/examples/language-modeling/output/checkpoint-3000000", tokenizer='roberta-base', device=0) #our finetuned model
tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')

In [5]:
form_emb = []
for form in tqdm(list_formula[:100000]):
        feat = feat_ext(form)[0]
        ans_emb = np.mean(feat, axis=0)
        form_emb.append(ans_emb)

In [6]:
form_emb = np.array(form_emb)

In [7]:
form_emb.shape

(100000, 768)

In [8]:
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(form_emb)

In [9]:
distances, indices = nbrs.kneighbors(form_emb[:20])

In [10]:
for i,nn in enumerate(indices[:20]):
#     print(list_formula[i])
    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px; margin-left: 15px">' +  f'${list_formula[i]}$ </div>'))
    for j in nn:
#         print(list_formula[i], list_formula[j])
        display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px; margin-left: 15px">' +  f'${list_formula[j]}$ </div>'))
    print('=========================================')

## evaluate NTCIR-12 Dataset ?

to know how good are these embeddings ? we should do that 😊

In [12]:
import jsonlines
from tqdm.notebook import tqdm
import re
import glob, os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from transformers import pipeline
from transformers import *
import math
from IPython.core.display import display, HTML


path_data = '/data/szr207/projects/Mathseer/output.jsonl'
list_exp = []
list_id = []
dict_id = {}
list_exp_orig = []

with jsonlines.open(path_data) as reader:
    for obj in tqdm(reader.iter(type=dict, skip_invalid=True)):
        list_exp.append(obj['exp'].rstrip().replace('%',' '))
#         list_exp_orig.append(obj['exp'].replace('%',' '))
        list_id.append(obj['id'])
        dict_id[obj['id']] = obj["list_id"]

In [16]:
def intersection(lst1, lst2): 
    lst3 = [value for value in tqdm(lst1) if value in lst2] 
    return lst3 

def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [17]:
len(intersection(list_exp, list_formula))

KeyboardInterrupt: 

In [19]:
len(list_formula)

10774550

In [17]:
from trectools import TrecRun, TrecEval, fusion, TrecQrel

r1 = TrecRun("/data/szr207/projects/Mathseer/bpe/new-concrete.dat")
qrels = TrecQrel('/data/szr207/projects/Mathseer/bpe/judge.dat')
r3 = TrecRun("/data/szr207/projects/Mathseer/bpe/res_tangent_cft")

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r3])
# r1_p25 = TrecEval(r1, qrels).get_bpref()          # P@25: 0.3392
# r2_p25 = TrecEval(r3, qrels).get_bpref()          # P@25: 0.2872
fused_run_p25 = TrecEval(fused_run, qrels).get_bpref()   # P@25: 0.3436

# print("P@25 -- Run 1: %.3f, Run 2: %.3f, Fusion Run: %.3f" % (r1_p25, r2_p25, fused_run_p25))

# Save run to disk with all its topics
fused_run.print_subset("rrf.cft.bert.txt", topics=fused_run.topics())

File rrf.cft.bert.txt writen.


In [18]:
fused_run_p25

0.643366279975944

In [15]:
TrecEval(r1, qrels).get_bpref()

0.4187256931858488

In [6]:
TrecEval(r3, qrels).get_bpref()

TypeError: '<' not supported between instances of 'float' and 'str'

In [12]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -l4 -m bpref /data/szr207/projects/Mathseer/bpe/judge.dat /data/szr207/projects/Mathseer/bpe/res_tangent_cft

bpref                 	all	0.6320
